In [1]:
from google.colab import drive
import os
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import tensorflow as tf
import numpy as np
import glob
from sklearn.model_selection import train_test_split
from PIL import Image

drive.mount('/content/drive')

os.chdir("/content/drive/My Drive/Poisson_Study/data_large")

Mounted at /content/drive


In [2]:

df = pd.read_csv("/content/drive/MyDrive/Poisson_Study/labels.csv")

import tensorflow as tf

class Block(tf.keras.Model):
    def __init__(self, filters, kernel_size, repetitions, pool_size=2, strides=2):
        super(Block, self).__init__()
        self.filters = filters
        self.kernel_size = kernel_size
        self.repetitions = repetitions
        self.conv_layers = [
            tf.keras.layers.Conv2D(filters, kernel_size, activation='relu', padding="same")
            for _ in range(repetitions)
        ]
        self.max_pool = tf.keras.layers.MaxPooling2D(pool_size, strides=strides)

    def call(self, inputs):
        x = inputs
        for conv in self.conv_layers:
            x = conv(x)
        return self.max_pool(x)

class MyVGG(tf.keras.Model):
    def __init__(self, num_classes):
        super(MyVGG, self).__init__()
        self.block_a = Block(filters=64, kernel_size=3, repetitions=2)
        self.block_b = Block(filters=128, kernel_size=3, repetitions=2)
        self.block_c = Block(filters=256, kernel_size=3, repetitions=3)
        self.block_d = Block(filters=512, kernel_size=3, repetitions=3)
        self.block_e = Block(filters=512, kernel_size=3, repetitions=3)
        self.flatten = tf.keras.layers.Flatten()
        self.fc = tf.keras.layers.Dense(256, activation='relu')
        self.classifier = tf.keras.layers.Dense(num_classes, activation='softmax')

    def call(self, inputs, return_last_conv=False):
        x = self.block_a(inputs)
        x = self.block_b(x)
        x = self.block_c(x)
        x = self.block_d(x)
        x = self.block_e(x)  # Last convolutional output
        if return_last_conv:
            return x
        x = self.flatten(x)
        x = self.fc(x)
        return self.classifier(x)




def call(self, inputs):
        x = self.block_a(inputs)
        x = self.block_b(x)
        x = self.block_c(x)
        x = self.block_d(x)
        x = self.block_e(x)
        x = self.flatten(x)
        x = self.fc(x)
        x = self.classifier(x)
        return x

def prepare_images(file_paths, image_size=(256, 256)):
    img_array = []
    labels = []
    for file_path in file_paths:
        base_name = os.path.basename(file_path)

        img = cv2.imread(file_path, cv2.IMREAD_GRAYSCALE)
        if img is not None:
            img = cv2.resize(img, image_size)
            img = np.expand_dims(img, axis=-1)  # Add channel dimension for grayscale
            img_array.append(img)

            label_series = df[df['filename'] == base_name]['hemorrhage']
            if not label_series.empty:
                labels.append(label_series.iloc[0])
            else:
                print(f"No label found for {base_name}; skipping image.")
                continue  # Skip this image if no label found
        else:
            print("Failed to load image:", file_path)

    return np.array(img_array) / 255.0, np.array(labels)



file_paths = glob.glob("/content/drive/My Drive/Poisson_Study/data_large/*.png")

X, y = prepare_images(file_paths)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=42)

model = MyVGG(num_classes=2)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.fit(X_train, y_train, epochs=1, validation_data=(X_test, y_test))


107/107 [==============================] - 96s 621ms/step - loss: 1.3880 - accuracy: 0.5066 - val_loss: 0.6931 - val_accuracy: 0.5117


In [3]:
model.save_weights("/content/drive/MyDrive/Poisson_Study/vgg.h5")
y_pred = model.predict(X_test)


19/19 [==============================] - 3s 141ms/step


In [4]:
import numpy as np
from sklearn.metrics import accuracy_score, classification_report

y_pred = model.predict(X_test)
y_pred_binary = [np.argmax(a) for a in y_pred]

print("Accuracy of model = ", accuracy_score(y_test, y_pred_binary))
print(classification_report(y_test, y_pred_binary, target_names=['0', '1']))



19/19 [==============================] - 2s 136ms/step
Accuracy of model =  0.5116666666666667
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       293
           1       0.51      1.00      0.68       307

    accuracy                           0.51       600
   macro avg       0.26      0.50      0.34       600
weighted avg       0.26      0.51      0.35       600



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
